In [1]:
## 에러가 뜰 경우 주석 때문입니다. 지우고 코드를 돌려야 합니다.

# !pip install pyperclip==1.8.2
# !pip install thefuzz==0.19.0
# !pip install requests==2.26.0
# !pip install selenium==4.1.0
# !pip install more_itertools==8.14.0
# !pip install bs4==4.4.1 #: BeautifulSoup 패키지 (만약 에러 뜰 경우 bs4로만 install.)
# !pip install html_table_parser #: 웹에서 테이블 추출하는 패키지

In [6]:
#패키지
import os
import re
import pyperclip
import pandas as pd
import numpy as np
import requests
from numpy import nan
import time
import statistics
from thefuzz import fuzz

import warnings
warnings.filterwarnings(action='ignore')

#change working dir.
os.chdir('E:/루당탕사라')

# 기업명 통합 코드
기업 관련 데이터를 사용할 때, 데이터 프레임마다 기업명 변수가 제각기일 수 있다. (ex. SKT, 에스케이티)  
특히 영어를 사용하는 기업이 이런 경우가 빈번한데, 이는 여러 데이터셋을 merge할 때 여러 문제를 야기할 수 있기 때문에 기업명(영어로 쓰인)을 통합하기 위한 코드를 작성하고자 한다.    

순서는 다음과 같다:
1. 가능한 기업명 후보군을 최대한 많이 생성한다. (이는 검색 등을 통해 데이터를 크롤링할 때 기업명 표기를 정확하게 사용하지 않으면 기업 정보가 뜨지 않는 경우가 있으므로 여러 후보군을 만들고자 한다.)  
    1-a. 영어->한국어로 번역한다.   
    1-b. 알파벳->한글발음으로 표기한다.    
    1-c. 영어->한글발음으로 표기한다.   
  
  
2. 한 기업 당 최대 3개의 이름이 있는 데이터 프레임을 생성한다.  
3. fuzz matching을 사용한 기업명 매칭(ind_name matching) 함수를 생성한다: match_name  
4. match_name 함수를 적용하여 데이터프레임의 기업명을 통합한다.

## 함수생성 1: 영어→한국어 번역
#### ex) naver -> 네이버로 번역하는 함수
- 파파고 API 사용: 아래 주소로 접속하여 파파고 오픈 API 이용 신청하여, 발급된 Client ID, Client Secret을 사용하였다.

###### 22.10.14일 기준으로 여전히 API 호출 작동됨.

In [7]:
def get_translate(text):
    
    client_id = " " # Client ID
    client_secret = " " # Client Secret

    data = {'text' : text,
            'source' : 'en',
            'target': 'ko'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

## 함수생성 2. 알파벳→한글표기 변환 함수
#### ex. SKT -> 에스케이티로 바꾸는 함수

In [8]:
def start_trans(word):
    ENGS = ['a', 'A', 'b', 'B', 'c', 'C', 'd', 'D', 'e', 'E', 'f', 'F', 'g', 'G', 'h', 'H', 'i', 'I',
            'j', 'J', 'k', 'K', 'l', 'L', 'm', 'M', 'n', 'N', 'o', 'O', 'p', 'P', 'q', 'Q', 'r', 'R',
            's', 'S', 't', 'T', 'u', 'U', 'v', 'V', 'w', 'W', 's', 'S', 'x', 'X', 'y', 'Y', 'z', 'Z', '&']
    KORS = ['에이', '에이', '비', '비', '씨', '씨', '디', '디', '이', '이', '에프', '에프', '지', '지',
            '에이치', '에이치', '아이', '아이', '제이', '제이', '케이', '케이', '엘', '엘', '엠', '엠',
            '엔', '엔', '오', '오', '피', '피', '큐', '큐', '알', '알', '에스', '에스', '티', '티',
            '유', '유', '브이', '브이', '더블유', '더블유', '에스', '에스', '엑스', '엑스', '와이', '와이', '지', '지', '앤']

    trans = dict(zip(ENGS, KORS))
    is_english = re.compile('[&a-zA-Z]')
    result = ''

    for i in word:
        temp = is_english.findall(i)
        if len(temp)>0:
            result = str(result)+str(trans[i])
        else:
            result = str(result)+str(i)

    return(result)

In [9]:
# test
print(get_translate('Sarah.'))
print(start_trans('SK'))

새라입니다.
에스케이


## 함수를 사용한 기업명 변환

In [13]:
code_df = pd.read_excel("종목코드.xlsx", dtype='object') #종목코드 데이터 load
name_list = code_df['종목명'] #종목명 list로 추출

In [14]:
translate_name = []
ko_name = []

trans_name = []
hangul = re.compile('[가-힣]+')

for i, name in enumerate(name_list):
    i+=1
    if (i % 100) == 0: print(str(i)+'번째 기업 변환 중...')
    if i==len(name_list): print('변환 종료.')
    
    #한글로 시작하는가? (한글로 시작하는 단어들은 파파고를 돌릴 필요가 없다.)
    ## 예외: NICE (좋아. 라고 번역되므로 직접 나이스로 수정하였다.)
    if len(hangul.findall(name[0]))>0:
        
        translate_name.append(name) 
        ko_name.append(start_trans(name))
        trans_name.append(start_trans(name))
    
    #만약 영어로 시작한다면 파파고 번역을 거친다.
    else: 
        new_name = get_translate(name)
        translate_name.append(new_name)
        ko_name.append(start_trans(name))
        trans_name.append(start_trans(new_name))

100번째 기업 변환 중...
200번째 기업 변환 중...
300번째 기업 변환 중...
400번째 기업 변환 중...
500번째 기업 변환 중...
600번째 기업 변환 중...
700번째 기업 변환 중...
800번째 기업 변환 중...
900번째 기업 변환 중...
변환 종료.


## 파일 생성 1: industry_name_최종.csv
#### 기업명 매칭 기준 1이 되는 파일

In [15]:
industry_name = pd.DataFrame({'name':name_list, 'translate_name':translate_name, 'ko_name':ko_name, 'trans_name':trans_name})
print(industry_name.head())

industry_name.to_csv("E:/루당탕사라/기업명매칭/industry_name.csv", index=None, encoding="cp949")

     name translate_name   ko_name trans_name
0  AJ네트웍스         AJ네트웍스  에이제이네트웍스   에이제이네트웍스
1   AK홀딩스          AK홀딩스   에이케이홀딩스    에이케이홀딩스
2   AP시스템          AP시스템    에이피시스템     에이피시스템
3     BGF            BGF      비지에프       비지에프
4  BGF리테일         BGF리테일   비지에프리테일    비지에프리테일


총 3개의 col이 형성되었다. 이 데이터 프레임은 앞으로 기업명을 검색하거나 매칭할 때 사용될 것이다.  
추가 작업: 파일을 열고 직접 기업명을 확인하며 잘못 변환된 것을 수정하고 추가적인 기업명 후보군을 삽입하였다.

##### [예시]
- E1: 이원
- 브이: 비 (티브이, 시지브이)
- N은 엔이라고 발음되기도 하지만 앤이라고도 한다.

-----

## 파일 생성 2: ind_name_with_CODE.xlsx
industry_name_최종.csv을 수정하여 생성하였다.

기준 기업명에 대해, translate_name, ko_name, trans_name을 중복 없이 모두 하나의 column에 삽입하고자 한다. (효율을 위함)  
또한 종목코드를 포함하여 전처리의 편의를 높이고자 했다.  
index column도 생성하여 기준 기업명의 위치를 unique하게 구분할 때 사용하였다.

In [14]:
ind_name = pd.read_csv('E:/루당탕사라/기업명매칭/industry_name_최종.csv', encoding='ANSI')
ind_name.head()

,name,translate_name,ko_name,trans_name
0,흥아해운,흥아해운,흥아해운,흥아해운
1,흥국화재,흥국화재,흥국화재,흥국화재
2,휴켐스,휴켐스,휴켐스,휴켐스
3,휴젤,휴젤,휴젤,휴젤
4,휴온스,휴온스,휴온스,휴온스


In [15]:
result = pd.DataFrame({'index':pd.Series(dtype='int'), 'name':[], 'trans_name':[]})

for index, row in ind_name.iterrows():
    row_set  = set(row)
    row_set.discard(np.nan)
    for trans_name in list(row_set):
        result.loc[len(result)] = [int(index), row[0], trans_name]

In [16]:
# csv 파일에 종목코드 merge 

result = pd.merge(code_df, result, how='right', right_on='name', left_on='종목명')
result.drop('종목명', axis=1, inplace=True)

result.to_excel("E:/루당탕사라/기업명매칭/ind_name_with_CODE.xlsx", index=None, encoding="cp949")

------------------------------

# ind_name matching 함수 정의
위에서 생성한 파일을 이용해 ind_name을 matching 하는 함수를 생성한다.: match_name


이 matching 함수는 따로 .py 파일로 저장하지 않고 코드를 직접 옮겨 사용한다. 그 이유는 데이터 특성에 맞춰 조금씩 코드를 수정해야 하는 상황이 발생했기 때문이다. 따라서 아래 코드는 다른 크롤링 및 전처리 코드 파일에 많이 등장할 예정이다. 

In [11]:
# 기업명-matching에 필요한 함수 정의

def match_name(file, col_name, file_type='excel', sheet_num=0):
    
    #기업명 csv 불러오기 (기준 파일)
    ind_name = pd.read_excel("기업명매칭/ind_name_with_CODE.xlsx", dtype='object')
    
    # 0. Load data file.
    if file_type == 'excel':
        data = pd.read_excel(file, sheet_name = sheet_num)
    
    elif file_type == 'csv':
        data = pd.read_csv(file, encoding='cp949')
    
    else:
        raise Exception("file_type must be 'excel' or 'csv'.")
    
    
    # 1. Put columns = [CODE, 실제 기업명(=ind_name)]
    data.insert(0, '종목코드', -1)
    data.insert(1, '회사명', -1)
      ## -1 = 해당 업체명은 우리가 가진 상장기업들에 속하지 않는 이름입니다.
    
    # 2. (주), 주식회사 등 불필요한 단어를 기준으로 split하고 다시 이어붙인다.
    ##필요에 따라 본부, 공장 등의 불필요한 단어를 포함한 단어들을 제거한다. (삭제 단어목록: remove_word)
    new_list = []
    remove_word = ['본부', '공장', '사업장', '캠퍼스', '발전소', '발전처']
    
    for name in data[col_name]:
        name = [i for i in re.split('\(주\)|㈜| |\xa0|주식회사', name) if not i==""]
        if len(name) > 1:
            new_list.append(''.join([x for x in name if not any(word in x for word in remove_word)]))
        else:
            new_list.append(name[0])
    
    # 4. 그렇게 생성된 name_list와 우리가 가진 기업명(ind_name)을 매칭하여 기업명을 부여한다.
    
        ## 매칭은 다음과 같은 단계로 이루어진다.
        # Step 0. name_list의 i번째 이름에 대해, 우리가 가진 모든 기업명(변환된 기업명 포함)과의 매칭 점수를 구한다: score
        
        # Step 1. name_list의 i번째 이름(=환경정보공개시스템의 각 사업장 명)에 대해, 포함되는 ind_name을 가져온다.
        #        (ex. name_list의 i번째 이름: '포스코 포천 사업장' => 결과값: 'POSCO')
        
        # Step 2. 만약 포함되는 ind_name이 1개라면 그값이 최종적으로 매칭된 기업명이 된다. (매칭 종료.)
        #        포함되는 ind_name이 0개라면 대상 기업명이 아니라고 간주하여 pass한다. (매칭 종료, -1 유지.)
        #       그러나 포함되는 ind_name이 2개 이상이라면, Step 3으로 이동한다.
        
        # Step 3. 포함되는 inde_name 중에서 가장  큰 매칭 점수(score)를 가지는 ind_name을 반환한다. (매칭 종료.)
    
    
    for i, name in enumerate(new_list):
    
        ## 매칭 점수 부여 (using fuzz matching: token_sort, token_set_ratio)
        score = [fuzz.token_sort_ratio(name, key) for key in ind_name['trans_name']]
        ## 해당 기업명이 ind_name 중에 하나를 포함하는지의 여부 
        contain_index = [row[0] for row in ind_name.iterrows() if row[1]['trans_name'] in name]
        contain_name = [key for key in ind_name['trans_name'] if key in name]

        if len(contain_name) == 1: ## 포함 기준으로 가져온다.
            data.loc[i, '종목코드'] = ind_name.loc[contain_index[0], 'CODE']
            data.loc[i, '회사명'] = ind_name.loc[contain_index[0], 'name']

        elif len(contain_name) == 0: ## 만약 포함되는 단어가 없을 경우, pass. (기업명 없다고 간주)
            continue

        else: ## 만약 여러개 ind_name을 포함할 경우, 그 중 가장 큰 score를 준 ind_name을 부여한다.
            score = [score[x] for x in contain_index]
            data.loc[i, '종목코드'] = ind_name['CODE'][contain_index[score.index(max(score))]]
            data.loc[i, '회사명'] = ind_name['name'][contain_index[score.index(max(score))]]

    return(data)

# 실데이터에 함수 적용

## 환경 데이터

In [36]:
start = time.time()
file_name = '환경마크인증_2020.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='업체명', file_type='excel', sheet_num=1)
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

In [37]:
start = time.time()
file_name = '환경마크인증_2021.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='업체명', file_type='excel', sheet_num=0)
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 5204.602179050446


In [23]:
start = time.time()
file_name = 'NGMS온실가스_2020.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='관리업체', file_type='excel', sheet_num=1)
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 57.75535774230957


In [25]:
start = time.time()
file_name = 'NGMS온실가스_2021.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='관리업체', file_type='excel', sheet_num=1)
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 58.67118978500366


In [32]:
start = time.time()
file_name = '환경부녹색기업3y.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='기업명')
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 5.565138816833496


In [33]:
start = time.time()
file_name = '환경성적표지인증_2020.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='기업명')
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 73.48806953430176


In [34]:
start = time.time()
file_name = '환경성적표지인증_2021.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='기업명')
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 81.37743544578552


In [35]:
start = time.time()
file_name = '환경법규위반_2021.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='기업명')
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 61.47468638420105


In [7]:
start = time.time()
file_name = 'CLEANSYS대기오염물질_2020.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='사업장명')
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 34.729243993759155


In [29]:
start = time.time()
file_name = 'CLEANSYS대기오염물질_2021.xlsx'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='사업장명')
data.to_excel("기업명매칭/기업명매칭 후/"+file_name, index=None, encoding='cp949')
print("time :", time.time() - start)

time : 45.099130392074585


### 환경데이터: 데이터 병합

In [30]:
df1 = pd.read_excel("기업명매칭/기업명매칭 후/CLEANSYS대기오염물질_2020.xlsx")
df2 = pd.read_excel("기업명매칭/기업명매칭 후/CLEANSYS대기오염물질_2021.xlsx")

df = pd.concat([df1, df2])
df.to_excel('기업명매칭/기업명매칭 후/대기오염물질2y_2021.xlsx', index=None, encoding='cp949')

In [26]:
df1 = pd.read_excel("기업명매칭/기업명매칭 후/NGMS온실가스_2020.xlsx")
df2 = pd.read_excel("기업명매칭/기업명매칭 후/NGMS온실가스_2021.xlsx")

df = pd.concat([df1, df2])
df.to_excel('기업명매칭/기업명매칭 후/온실가스2y_2021.xlsx', index=None, encoding='cp949')

In [38]:
df1 = pd.read_excel("기업명매칭/기업명매칭 후/환경마크인증_2020.xlsx")
df2 = pd.read_excel("기업명매칭/기업명매칭 후/환경마크인증_2021.xlsx")

df = pd.concat([df1, df2])
df.to_excel('기업명매칭/기업명매칭 후/환경마크인증2y_2021.xlsx', index=None, encoding='cp949')

## 사회 데이터

In [14]:
start = time.time()
file_name = '소비자중심경영인증기업.csv'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='법인명', file_type='csv')
data.to_excel("기업명매칭/기업명매칭 후/소비자중심경영인증기업_2021.xlsx", index=None, encoding='cp949')
print("time :", time.time() - start)

time : 11.135126113891602


In [15]:
start = time.time()
file_name = '한국사회복지협의회_지역사회공헌 인정기업기관_20220131.csv'
data = match_name(file = "기업명매칭/기업명매칭 전/"+file_name, col_name='기관명', file_type='csv')
data.to_excel("기업명매칭/기업명매칭 후/지역사회공헌인증_2021.xlsx", index=None, encoding='cp949')
print("time :", time.time() - start)

time : 18.340946197509766


#### 이렇게 생성된 파일들은 모두 기업명 매칭이 잘 되었는지 확인되었으며, 매칭이 잘 안된 기업은 수기로 변경해주었다.